# Generates the mt5-small test embeddings

In [ ]:
pip install --user torch

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user datasets

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user transformers

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user sentence_transformers

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --user fsspec

You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig,T5Tokenizer
from transformers import pipeline, AutoTokenizer

In [ ]:
import argparse

from sentence_transformers import SentenceTransformer, models
from tqdm import tqdm
import datasets

import torch
from transformers import TranslationPipeline, FeatureExtractionPipeline, MT5EncoderModel, AutoConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers.pipelines.pt_utils import KeyDataset

#from models.model_mT5_contrastive import mT5Classifier

In [ ]:
import fsspec

In [ ]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any
from tqdm import tqdm
import torch
import torch.nn as nn
import pandas as pd
import pickle
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from numpy import save
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
import datasets

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization

from sklearn.model_selection import train_test_split

In [ ]:
import datasets

import torch
from transformers import TranslationPipeline, FeatureExtractionPipeline, MT5EncoderModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers.pipelines.pt_utils import KeyDataset

In [ ]:
df = pd.read_csv('Data/test_google_translate.csv')

In [ ]:
df.head()

,ID,Text
0,ID_ADHEtjTi,Abambo odzikhweza akuchuluka Kafukufuku wa ap...
1,ID_AHfJktdQ,Ambuye Ziyaye Ayamikira Aphunzitsi a Tilitonse...
2,ID_AUJIHpZr,Anatcheleza: Akundiopseza a gogo wanga Akundi...
3,ID_AUKYBbIM,Ulova wafika posauzana Adatenga digiri ya uph...
4,ID_AZnsVPEi,"Dzombe kukoma, koma Kuyambira makedzana, pant..."


In [ ]:
def embed_sentence(in_path: str, out_path: str, finetuned: bool=False, batch_size:int=32):
    # tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", model_max_length=512)
    # model = MT5EncoderModel.from_pretrained("google/mt5-small")
    # pipe = FeatureExtractionPipeline(model, tokenizer=tokenizer, device=-1, batch_size=batch_size)
    if finetuned:
        model = mT5Classifier(load_model_path="dump/mt5_sup_chi")
    else:
        model_name = "google/mt5-small"
        base_model = models.Transformer(model_name)
        # sentence bert repo does not current support mT5 yet (only T5 ironically)
        MT5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
        mT5_config = AutoConfig.from_pretrained(model_name)
        base_model.auto_model = MT5EncoderModel.from_pretrained("google/mt5-small", config=mT5_config)
        pooling_model = models.Pooling(base_model.get_word_embedding_dimension())
        model = SentenceTransformer(modules=[base_model, pooling_model])

    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    kwargs = {'truncation': True, 'max_length': 512, 'return_text': True, "clean_up_tokenization_spaces": True}
    if in_path.endswith(".txt"):
        dataset = datasets.load_dataset('text', data_files={'train': [in_path]})["train"]
    elif in_path.endswith(".csv"):
        dataset = datasets.load_dataset('csv', data_files={'train': [in_path]})["train"]
        dataset = dataset.rename_column("Text", "text")
    elif ".jsonl" in in_path:
        dataset = datasets.load_dataset('text', data_files={'train': [in_path]})["train"]
        dataset = dataset.map(lambda example: {"content": json.loads(example["text"])["text"]}, remove_columns=["text"])
        dataset = dataset.rename_column("content", "text")
    else:
        raise NotImplementedError()

    # pbar = tqdm(total=len(dataset))
    # embeddings = np.zeros([len(dataset), model.config.hidden_size if not finetuned else 256])
    # for i, out in enumerate(pipe(KeyDataset(dataset, "text"), **kwargs)):
        # embeddings[i] = np.array(out[0][:512]).mean(1)
        # pbar.update(1)
    embeddings = model.encode(dataset["text"], batch_size=batch_size, show_progress_bar=True, device="cpu")
    np.save(out_path, embeddings)


In [ ]:
embed_sentence('Data/Test.csv','Data/embeddings_chichewa_512_final_test.npy', finetuned=False, batch_size=2)

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MT5EncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5EncoderModel from the 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /usr4/cs640g/kmn5409/.cache/huggingface/datasets/csv/default-e6c1acd721aff5d1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/310 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
embds_ = np.load('Data/embeddings_chichewa_512_final_test.npy')

In [ ]:
embds_.shape

(620, 512)

In [ ]:
embds_[0]

array([ 2.14366857e-02, -1.13247745e-01,  3.79235931e-02, -1.81340486e-01,
       -5.57690226e-02,  4.92046699e-02,  2.68003792e-01,  5.31835891e-02,
        5.77673391e-02, -7.49067664e-02,  7.53107890e-02,  3.15462094e-04,
       -1.13170564e-01, -1.24731414e-01,  1.32339016e-01, -8.21345150e-02,
       -1.57658279e-01, -5.35773486e-02, -4.70860861e-02,  1.53217360e-01,
        1.87997833e-01,  9.15086940e-02,  6.83001056e-02,  1.34871919e-02,
       -2.56842434e-01, -1.04348511e-01, -6.94524050e-01,  9.55336634e-03,
        1.60659358e-01,  1.85761452e-01,  1.45093799e-01,  2.91559435e-02,
       -3.82554382e-02, -4.85149510e-02, -1.19893225e-02,  1.37504801e-04,
        8.45603943e-02,  1.38507620e-01,  2.97549397e-01, -1.69776633e-01,
        2.01809704e-01,  3.62301394e-02, -9.03215185e-02,  1.17907092e-01,
        1.64633051e-01, -1.36115670e-01,  1.51187062e-01, -6.53492869e-04,
        5.73066622e-03, -8.96793008e-02, -8.90804268e-03,  2.45660469e-02,
        1.88104790e-02,  

In [ ]:
embed_sentence('Data/test_google_translate.csv','Data/embeddings_english_512_final_test.npy', finetuned=False, batch_size=2)

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5EncoderModel: ['lm_head.weight']
- This IS expected if you are initializing MT5EncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5EncoderModel from the 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /usr4/cs640g/kmn5409/.cache/huggingface/datasets/csv/default-042f9ac6e8e84afb/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/310 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
embds_english = np.load('Data/embeddings_english_512_final_test.npy')

In [ ]:
embds_english.shape

(620, 512)

In [ ]:
embds_english[0]

array([ 1.26552641e-01, -4.36130166e-02, -1.95987197e-03, -3.77837718e-02,
       -5.71503565e-02,  9.07313228e-02,  2.84052312e-01,  8.82943068e-03,
       -2.53661722e-02, -1.79747909e-01,  8.78747031e-02, -6.73876777e-02,
       -1.97384074e-01, -6.96042776e-02,  2.18795165e-01, -1.20242536e-01,
        1.47524491e-01, -3.87766846e-02, -4.34632152e-02,  1.63378745e-01,
        1.73909083e-01,  8.20972919e-02,  1.99337557e-01,  1.42802298e-01,
        4.23377343e-02, -1.43944442e-01, -6.88462019e-01, -5.21189980e-02,
       -1.46165401e-01,  1.14073254e-01,  7.51407295e-02, -2.48527508e-02,
        1.78471245e-02, -3.17766219e-02, -3.42976861e-03, -1.27798572e-01,
       -5.81632741e-02,  1.33321524e-01,  4.38067168e-01, -1.44747153e-01,
        1.39805347e-01,  1.16010986e-01,  8.81974101e-02,  2.15058908e-01,
        1.37229159e-01,  1.15452535e-01,  1.14660345e-01, -8.45719427e-02,
        5.89411035e-02, -5.85021563e-02,  2.18726844e-02, -1.11078165e-01,
        4.95406389e-02,  